# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [2]:
#Install the dependencies
#%%capture
!pip install dlt[duckdb]

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [4]:
# to do: homework :)
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

generator = square_root_generator(5)
sum = 0

for sqrt_value in generator:
    sum+=sqrt_value

print('Question 1:', sum)

generator = square_root_generator(13)

for sqrt_value in generator:
    pass

print('Question 2:', sqrt_value)

Question 1: 8.382332347441762
Question 2: 3.605551275463989


In [6]:
import dlt

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

pipeline = dlt.pipeline(pipeline_name="workshop_homework",
						destination='duckdb', 
						dataset_name='people_data')

info = pipeline.run(people_1(), 
                    table_name="people", 
                    write_disposition="replace")

print(info)

Pipeline workshop_homework load step completed in 0.68 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////home/ih/workspace/workshop_homework.duckdb location to store data
Load package 1707344950.5917072 is LOADED and contains no failed jobs


In [7]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n People table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 People table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707344950.5917072,H35xY45Lj13upg
1,2,Person_2,27,City_A,1707344950.5917072,HI/W6fvAenOLvg
2,3,Person_3,28,City_A,1707344950.5917072,8T5bTesPCclxGg
3,4,Person_4,29,City_A,1707344950.5917072,9y5MjAq7YSd4lQ
4,5,Person_5,30,City_A,1707344950.5917072,uA7PvSUVLdyJRA


In [ ]:
info = pipeline.run(people_2(), 
                    table_name="people", 
                    write_disposition="append")

print(info)

In [11]:
print("\n\n\n People table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

sum = conn.sql("SELECT SUM(age) FROM people").df()
print('Question 3:')
display(sum)




 People table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707344950.5917072,H35xY45Lj13upg,None
1,2,Person_2,27,City_A,1707344950.5917072,HI/W6fvAenOLvg,None
2,3,Person_3,28,City_A,1707344950.5917072,8T5bTesPCclxGg,None
3,4,Person_4,29,City_A,1707344950.5917072,9y5MjAq7YSd4lQ,None
4,5,Person_5,30,City_A,1707344950.5917072,uA7PvSUVLdyJRA,None
5,3,Person_3,33,City_B,1707345125.8603132,d+7+vH4vmOAWwQ,Job_3
6,4,Person_4,34,City_B,1707345125.8603132,A0lii61lJpbXGw,Job_4
7,5,Person_5,35,City_B,1707345125.8603132,PHj4bY22IKH27Q,Job_5
8,6,Person_6,36,City_B,1707345125.8603132,i+qv8muicUJoYg,Job_6
9,7,Person_7,37,City_B,1707345125.8603132,TgzqchmI9C7gtQ,Job_7


Question 3:


,sum(age)
0,353.0


In [33]:
info = pipeline.run(people_1(), 
                    table_name="people_merged", 
                    write_disposition="replace")

print(info)

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n People_merged table below:")

people_merged = conn.sql("SELECT * FROM people_merged").df()
display(people_merged)

Pipeline workshop_homework load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////home/ih/workspace/workshop_homework.duckdb location to store data
Load package 1707346656.923195 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merged       │
│ people_merged_2     │
└─────────────────────┘




 People_merged table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707346656.923195,ENrYX9iQj89Pnw,None
1,2,Person_2,27,City_A,1707346656.923195,xKhPG/mmi9A/uQ,None
2,3,Person_3,28,City_A,1707346656.923195,kmCVn56zXpeWqA,None
3,4,Person_4,29,City_A,1707346656.923195,IEE6pg0TU920bQ,None
4,5,Person_5,30,City_A,1707346656.923195,/SsRrXC5WiSqpA,None


In [42]:
info = pipeline.run(people_2,
					table_name="people_merged",
					write_disposition="merge",
                    primary_key="id")

print(info)

print("\n\n\n People_merged table below:")

people_merged = conn.sql("SELECT * FROM people_merged").df()
display(people_merged)

Pipeline workshop_homework load step completed in 0.61 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////home/ih/workspace/workshop_homework.duckdb location to store data
Load package 1707347515.950844 is LOADED and contains no failed jobs



 People_merged table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707346656.923195,ENrYX9iQj89Pnw,None
1,2,Person_2,27,City_A,1707346656.923195,xKhPG/mmi9A/uQ,None
2,8,Person_8,38,City_B,1707347515.950844,GcoarPG27tD8Hg,Job_8
3,5,Person_5,35,City_B,1707347515.950844,01H2UBhh3CNxTg,Job_5
4,4,Person_4,34,City_B,1707347515.950844,NhdwJ6JvP0ywfg,Job_4
5,7,Person_7,37,City_B,1707347515.950844,6jHlzAP+3FpdfQ,Job_7
6,3,Person_3,33,City_B,1707347515.950844,MrMzzQz3Y1V46A,Job_3
7,6,Person_6,36,City_B,1707347515.950844,wEWIOmluFTMM8g,Job_6


In [43]:
sum = conn.sql("SELECT SUM(age) FROM people_merged").df()
print('Question 4:')
display(sum)

Question 4:


,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX